Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

Here's my method:
- Depth first search on the tree of possible moves each turn (build one of the 4 robots if you have the resources, or do nothing)
- That tree is big, so introduce two ways of pruning it: (idk how important the 2nd one is, but it only helps)
  1. At each step of the DFS, calculate the maximum number of geodes possible for a relaxed version of the problem (starting from the current state). If the number of geodes possible in the relaxed version is not more than the maximum number of geodes found so far for the original problem, there's no benefit to continuing this branch. This relaxed problem is:
    - Rather than choosing which of the 4 robots to build, allow ourselves to build all 4 at once
    - But this still runs into resource sharing issues making it hard to solve, so instead assume that there are 4 different resource piles (one for each robot) to avoid having to consider any tradeoffs
    - When resources are produced, a copy of the resource is added to each of the 4 resource piles. But when a robot is built, it only uses up resources from its resource pile.
    - Any sequence of actions (i.e. builds and waits) to the original problem can be done in this relaxed problem, so the number of geodes attainable in this problem is an upper bound for the original problem
    - But crucially this problem is easy to solve: since there's no resource tradeoffs to make between robots, at each minute it's optimal to build the corresponding robot for each resource pile (resources permitting).
    - So we can solve it with a quick recursion (instead of a DFS)
  2. At each step of the DFS, under optimal play the only valid reason to not build a robot is if we want to save up resources for a particular robot. That is, waiting means we get to accumulate resources from existing robots to be able to afford a new robot. Let's unpack what this implies:
    - Say a robot is "now buildable" if we have the resources to build it right now
    - Say a robot is "future buildable" if we have the robots to produce the resources to build this robot. So we might not be able to build this robot now, but if we just wait a few minutes we'll have the resources.
    - Note ore and clay robots are always future buildable, because we start with an ore robot. Obsidian and geode robots are initially not future buildable.
    - If all our future buildable robots are now buildable, there is no reason to wait this turn. Waiting accumulates more resources, but if we want to build a robot then waiting until next turn just means we miss out on one turn of this new robot producing resources.
    - So if all our future buildable robots are now buildable, remove the option of waiting this turn.
    - Reframed: if there is a robot that is not now buildable but it is future buildable, add the option of waiting this turn.
- The function DFS() does the bulk of the work, while DFS_op() is the recursive function solving the relaxed version.
- The data organization is pretty terse: ex. each input blueprint becomes a tuple (ore robot ore cost, clay robot ore cost, obsidian robot ore cost, obsidian robot clay cost, geode robot ore cost, geode robot obsidian cost). DFS() uses tuples with 4 elements for each resource, and DFS_op() uses the 6-element tuple format with an extra element for geodes. Navigating the indices takes some work.
- It took a few tries to get the right pruning function (1). At one point I had it but was using a < instead of <= comparison, so in cases where the max number of geodes was 0 the pruning was not kicking in! This meant it took 1.5 hours to run instead of 0.05 seconds.

## 1 test

Inputs have a standardized order, should be easy to parse.

Similar solution to the elephants in the volcano problem should work: DFS on the finite tree of decisions I can do. I'll have to make a somewhat clever overestimate for number of geodes cracked. I can also leverage monotonicity: if state 1 has at least as many robots + resources + time as state 2, then state 1 can crack at least as many geodes as state 2.

For each time step
- I can choose to build one robot: spend the resources
- All my robots produce 1 of their resource
- Gain the robot if I built one

This means at each minute I can choose one of the 4 robot types to build (assuming I have enough resources). But $4^{24} > 10^{14}$ is definitely an overestimate for the number of meaningful actions I can take, because it's very unlikely I'd have the resources to build every robot for most turns. Let's try running this as-is and see if I need to do anything fancy. FWIW WLOG if after gathering resources I can afford to build all the robots that can be built with the resource types I have, I *must* build a robot.

Basic DFS was taking too long. I'll add an overestimate of final geodes, and change the geode system to add all future geodes. Also when branching, prioritize building a geode robot.
- Basic overestimate: current geodes accounted for, plus geodes if we build a geode robot every turn from now on. If the current time is t, the robot this turn gives (t-1), then (t-2), etc. So in total we get at most $t(t-1)/2$ extra geodes.
- This works for the first test blueprint but not the second one :(
- More advanced: consider all the possible ore we could ever get as current + future from existing robots plus whatever we can get from constructing new robots. Then do the same for clay (using this total amount for the ore), etc.
- This still isn't good enough for the 2nd blueprint! It is pruning 74% of branches, but that might be at a pretty far depth and I'd want to prune earlier on.
- Tried pruning based on previously seen states, but I don't have an efficient implementation and I'm not convinced it would be better anyways.
- Let's try another version of the overestimate prune: imagine there are 4 separate resources piles (one for the robot of each type). So I can just solve the recursion (no DFS!) for that quickly, and use that as the pruning estimate for the DFS.
- This works really well! Nice! It's having a hard time with the first blueprint in the actual run though. I'll let it run for a while, and see if I can think up a tighter geode overestimate.
- I modified the DFS to only allow the Nothing case if we're trying to save up for something (since otherwise Nothing doesn't achieve anything), which helped for blueprint 1. But now it's stuck on blueprint 10.
- Interesting that it's stuck on 10, when 2 is the same but with slightly cheaper robots.

OH I'm so tilted. I let part 2 run and it took 1.5 hours. That's when I noticed some of them give 0 geodes. A quick check and there was no bug in my code, so that was a true 0. Which made me realize that parseQ has a < sign which really could be a <= sign. That change makes it run in 0.05 seconds.

In [65]:
from tqdm.notebook import tqdm

In [66]:
split(t)

['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.',
 'Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.']

In [67]:
def parse(blueprint):
    aux = blueprint.split()
    I = [6,12,18,21,27,30]
    return tuple([int(aux[i]) for i in I])

blueprints = [parse(blueprint) for blueprint in split(t)]
blueprints

[(4, 2, 3, 14, 2, 7), (2, 3, 3, 8, 3, 12)]

In [68]:
# resources: [ore, clay, obsidian, geode]
# robots: [ore, clay, obsidian, geode]
# t: time left
# blueprint
def DFS(resources, robots, t, blueprint):
    global max_geodes
    
    ## If time ran out, update the max geodes and exit
    if t == 0:
        max_geodes = max(max_geodes, resources[3])
        return
    
    ## Exit if we can prune
    if pruneQ(resources, robots, t, blueprint):
        return
    
    ## Produce resources
    produced_resources = resources.copy()
    for i in range(4):
        produced_resources[i] += robots[i]
    
    ## Pick something to do: nothing, or build a robot
    # When building a robot, make sure to use the resources from the start of this turn
    # Attempting the builds in "priority" order (geode, obsidian, clay, ore)
    # Only allow the Nothing case if we can do it to save up for something
    allow_nothing = False
    
    # Build geode
    if (resources[0] >= blueprint[4]) and (resources[2] >= blueprint[5]):
        # Charge the cost of building the robot
        new_resources = produced_resources.copy()
        new_resources[0] -= blueprint[4]
        new_resources[2] -= blueprint[5]
        
        # Create the new robot
        new_robots = robots.copy()
        new_robots[3] += 1
        
        # Launch the continuation of the DFS
        DFS(new_resources, new_robots, t-1, blueprint)
    # If didn't build a geode robot but we do have at least one obsidian robot, allow for the Nothing case to save up
    elif robots[2]:
        allow_nothing = True
        
    # Build obsidian
    if (resources[0] >= blueprint[2]) and (resources[1] >= blueprint[3]):
        new_resources = produced_resources.copy()
        new_resources[0] -= blueprint[2]
        new_resources[1] -= blueprint[3]
        new_robots = robots.copy()
        new_robots[2] += 1
        DFS(new_resources, new_robots, t-1, blueprint)
    elif robots[1]:
        allow_nothing = True
    
    # Build clay
    if resources[0] >= blueprint[1]:
        new_resources = produced_resources.copy()
        new_resources[0] -= blueprint[1]
        new_robots = robots.copy()
        new_robots[1] += 1
        DFS(new_resources, new_robots, t-1, blueprint)
    else:
        allow_nothing = True
        
    # Build ore
    if resources[0] >= blueprint[0]:
        new_resources = produced_resources.copy()
        new_resources[0] -= blueprint[0]
        new_robots = robots.copy()
        new_robots[0] += 1
        DFS(new_resources, new_robots, t-1, blueprint)
    else:
        allow_nothing = True
    
    # Build nothing only if we're letting ourselves save up for something
    if allow_nothing:
        DFS(produced_resources, robots, t-1, blueprint)

    
    
# resources: [ore's ore, clay's ore, obs's ore, obs's clay, geo's ore, geo's obs, geode]
# robots: [ore, clay, obsidian, geode]
# t: time left
# blueprint

# This is the "overpowered" version where each of my robot types has a separate resource pile, and I can build multiple robots per turn
def DFS_op(resources, robots, t, blueprint):
    
    # If time ran out, return
    if t == 0:
        return resources[-1]
    
    # Produce resources
    new_resources = resources.copy()
    new_resources[0] += robots[0]
    new_resources[1] += robots[0]
    new_resources[2] += robots[0]
    new_resources[4] += robots[0]
    new_resources[3] += robots[1]
    new_resources[5] += robots[2]
    new_resources[6] += robots[3]
    
    ## For each robot type, try to build a robot
    new_robots = robots.copy()
    
    # Ore
    if resources[0] >= blueprint[0]:
        new_resources[0] -= blueprint[0]
        new_robots[0] += 1
    
    # Clay
    if resources[1] >= blueprint[1]:
        new_resources[1] -= blueprint[1]
        new_robots[1] += 1
    
    # Obsidian
    if (resources[2] >= blueprint[2]) and (resources[3] >= blueprint[3]):
        new_resources[2] -= blueprint[2]
        new_resources[3] -= blueprint[3]
        new_robots[2] += 1
    
    # Geode
    if (resources[4] >= blueprint[4]) and (resources[5] >= blueprint[5]):
        new_resources[4] -= blueprint[4]
        new_resources[5] -= blueprint[5]
        new_robots[3] += 1
    
    # Return the result of the next step
    return DFS_op(new_resources, new_robots, t-1, blueprint)

def pruneQ(resources, robots, t, blueprint):
    global max_geodes
    return DFS_op([resources[0], resources[0], resources[0], resources[1], resources[0], resources[2], resources[3]], robots, t, blueprint) <= max_geodes

In [69]:
max_geodes = 0
DFS([0,0,0,0], [1,0,0,0], 24, blueprints[0])
max_geodes

9

In [70]:
max_geodes = 0
DFS([0,0,0,0], [1,0,0,0], 24, blueprints[1])
max_geodes

12

In [47]:
from tqdm.notebook import trange, tqdm

In [48]:
quality = 0
for i in tqdm(range(len(blueprints))):
    max_geodes = 0
    DFS([0,0,0,0], [1,0,0,0], 24, blueprints[i])
    quality += (i+1) * max_geodes
quality

  0%|          | 0/2 [00:00<?, ?it/s]

33

## 1 run

In [73]:
blueprints = [parse(blueprint) for blueprint in split(s)]
blueprints

[(4, 4, 4, 18, 4, 9),
 (2, 2, 2, 17, 2, 10),
 (4, 4, 2, 7, 4, 13),
 (4, 3, 4, 20, 4, 8),
 (4, 4, 4, 5, 2, 10),
 (4, 4, 4, 8, 3, 19),
 (4, 4, 4, 8, 4, 14),
 (4, 4, 3, 6, 2, 14),
 (3, 3, 3, 6, 2, 16),
 (2, 4, 4, 19, 2, 18),
 (3, 4, 4, 14, 4, 10),
 (2, 3, 3, 13, 3, 15),
 (3, 4, 4, 13, 3, 7),
 (2, 4, 4, 16, 4, 17),
 (3, 4, 3, 15, 3, 20),
 (4, 4, 2, 18, 4, 20),
 (2, 4, 4, 18, 2, 11),
 (3, 4, 2, 14, 3, 14),
 (4, 4, 2, 11, 2, 7),
 (4, 3, 2, 19, 3, 10),
 (4, 4, 4, 7, 2, 19),
 (2, 3, 3, 18, 2, 19),
 (4, 3, 4, 20, 2, 15),
 (2, 4, 4, 13, 3, 11),
 (3, 3, 3, 8, 2, 12),
 (2, 4, 2, 20, 3, 15),
 (3, 4, 4, 18, 3, 13),
 (4, 4, 4, 17, 2, 13),
 (2, 4, 3, 14, 4, 9),
 (3, 4, 4, 6, 3, 16)]

In [74]:
geodes = [-1] * len(blueprints)
for i in tqdm(range(len(blueprints))):
    max_geodes = 0
    DFS([0,0,0,0], [1,0,0,0], 24, blueprints[i])
    geodes[i] = max_geodes
geodes

  0%|          | 0/30 [00:00<?, ?it/s]

[0,
 9,
 3,
 1,
 5,
 0,
 1,
 3,
 7,
 0,
 2,
 5,
 4,
 1,
 0,
 0,
 2,
 1,
 3,
 1,
 1,
 1,
 0,
 5,
 7,
 1,
 0,
 0,
 6,
 5]

In [75]:
score = 0
for i in range(len(geodes)):
    score += (i+1) * geodes[i]
score

1115

In [72]:
max_geodes = 0
DFS([0,0,0,0], [1,0,0,0], 24, blueprints[0])
max_geodes

0

## 2 test

None needed

## 2 run

In [78]:
blueprints = [parse(blueprint) for blueprint in split(s)[:3]]
blueprints

[(4, 4, 4, 18, 4, 9), (2, 2, 2, 17, 2, 10), (4, 4, 2, 7, 4, 13)]

In [79]:
geodes = [-1] * len(blueprints)
for i in tqdm(range(len(blueprints))):
    max_geodes = 0
    DFS([0,0,0,0], [1,0,0,0], 32, blueprints[i])
    geodes[i] = max_geodes
geodes

  0%|          | 0/3 [00:00<?, ?it/s]

[16, 54, 29]

In [80]:
16*54*29

25056

# Utilities

In [4]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [5]:
t = """
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.
"""

In [6]:
s = """
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 18 clay. Each geode robot costs 4 ore and 9 obsidian.
...
Blueprint 30: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 6 clay. Each geode robot costs 3 ore and 16 obsidian.
"""